In [1]:
import gc
import os
import random
import time
import math

import cv2
import numpy as np
import matplotlib.pyplot as plt

# model
import torch
import torchvision
import torch.nn as nn
import timm
from torchvision.models.feature_extraction import create_feature_extractor
import torchvision.transforms.functional as TF

# data loader
import albumentations as A
from albumentations.pytorch import ToTensorV2
from torch.utils.data import DataLoader, Dataset

# training
from torch.optim import SGD, Adam, AdamW
from torch.optim.lr_scheduler import CosineAnnealingLR, CosineAnnealingWarmRestarts, ReduceLROnPlateau, ExponentialLR

# metric
from sklearn.metrics import fbeta_score, roc_auc_score

import wandb

# config

In [2]:
BASE_DIR = "/working/"
EXP_NAME = "exp014"
OUTPUT_DIR = os.path.join(BASE_DIR, "output", EXP_NAME)

# metric

In [3]:
def fbeta_numpy(targets, preds, beta=0.5, smooth=1e-5):
    y_true_count = targets.sum()
    ctp = preds[targets==1].sum()
    cfp = preds[targets==0].sum()
    beta_squared = beta * beta

    c_precision = ctp / (ctp + cfp + smooth)
    c_recall = ctp / (y_true_count + smooth)
    dice = (1 + beta_squared) * (c_precision * c_recall) / (beta_squared * c_precision + c_recall + smooth)

    return dice

def calc_fbeta_auc(mask, mask_pred):
    mask = mask.astype(int).flatten()
    mask_pred = mask_pred.flatten()

    best_th = 0
    best_dice = 0
    dice_list = [] 
    # for th in np.array(range(10, 50+1, 5)) / 100:
    for th in np.array(range(10, 100+1, 5)) / 100:
        # dice = fbeta_score(mask, (mask_pred >= th).astype(int), beta=0.5)
        dice = fbeta_numpy(mask, (mask_pred >= th).astype(int), beta=0.5)
        dice_list.append(dice)
        # print(f'\t th: {th}, fbeta: {dice}')
        if dice > best_dice:
            best_dice = dice
            best_th = th
    
    auc = roc_auc_score(mask, mask_pred)
    # Logger.info(f'best_th: {best_th}, fbeta: {best_dice}')
    return best_dice, best_th, auc, dice_list


def calc_cv(mask_gt, mask_pred):
    best_dice, best_th, auc, dice_list = calc_fbeta_auc(mask_gt, mask_pred)

    return best_dice, best_th, auc, dice_list

In [11]:
def opening(mask, kernel_size=30, iter=3):
    kernel = np.ones((kernel_size, kernel_size), np.uint8)
    mask = cv2.erode(mask, kernel, iterations=iter)
    mask = cv2.dilate(mask, kernel, iterations=iter)
    return mask

def closing(mask, kernel_size=30, iter=3):
    kernel = np.ones((kernel_size, kernel_size), np.uint8)
    mask = cv2.dilate(mask, kernel, iterations=iter)
    mask = cv2.erode(mask, kernel, iterations=iter)
    return mask

---
# OOF SCORE CHECK

In [5]:
for th_ in range(55, 80, 5):
    th = th_ / 100
    pred_flatten_list = []
    mask_flatten_list = []
    for fold in [0,1,2,3,4]:
        pred_path = os.path.join(OUTPUT_DIR, "imgs", f"fold{fold}_average_slice555_valid_pred_img.png")
        mask_path = os.path.join(OUTPUT_DIR, "imgs", f"fold{fold}_average_slice555_valid_targets_img.png")
        pred_img = cv2.imread(pred_path, cv2.IMREAD_GRAYSCALE)
        mask_img = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)

        pred_binary = (pred_img/255. >= th).astype(np.uint8)
        pred_binary = closing(pred_binary, 65)
        mask_flatten_list.extend((mask_img.flatten()/255.).astype(int))
        pred_flatten_list.extend(pred_binary.flatten())
    dice = fbeta_numpy(np.array(mask_flatten_list), np.array(pred_flatten_list), beta=0.5)
    print(f"th={th:.2f}, dice={dice:.4f}")
    

th=0.55, dice=0.4688
th=0.60, dice=0.4915
th=0.65, dice=0.5088
th=0.70, dice=0.5211
th=0.75, dice=0.5277
th=0.80, dice=0.5176
th=0.85, dice=0.4911
th=0.90, dice=0.4436
th=0.95, dice=0.3626


In [6]:
for th_ in range(55, 100, 5):
    th = th_ / 100
    pred_flatten_list = []
    mask_flatten_list = []
    for fold in [0,1,2,3,4]:
        pred_path = os.path.join(OUTPUT_DIR, "imgs", f"fold{fold}_average_slice555_valid_pred_img.png")
        mask_path = os.path.join(OUTPUT_DIR, "imgs", f"fold{fold}_average_slice555_valid_targets_img.png")
        pred_img = cv2.imread(pred_path, cv2.IMREAD_GRAYSCALE)
        mask_img = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)

        pred_binary = (pred_img/255. >= th).astype(np.uint8)
        pred_binary = closing(pred_binary, 30)
        mask_flatten_list.extend((mask_img.flatten()/255.).astype(int))
        pred_flatten_list.extend(pred_binary.flatten())
    dice = fbeta_numpy(np.array(mask_flatten_list), np.array(pred_flatten_list), beta=0.5)
    print(f"th={th:.2f}, dice={dice:.4f}")
    

th=0.55, dice=0.5057
th=0.60, dice=0.5232
th=0.65, dice=0.5331
th=0.70, dice=0.5363
th=0.75, dice=0.5295
th=0.80, dice=0.5101
th=0.85, dice=0.4746
th=0.90, dice=0.4227
th=0.95, dice=0.3409


In [7]:
for th_ in range(55, 80, 5):
    th = th_ / 100
    pred_flatten_list = []
    mask_flatten_list = []
    for fold in [0,1,2,3,4]:
        pred_path = os.path.join(OUTPUT_DIR, "imgs", f"fold{fold}_average_slice555_valid_pred_img.png")
        mask_path = os.path.join(OUTPUT_DIR, "imgs", f"fold{fold}_average_slice555_valid_targets_img.png")
        pred_img = cv2.imread(pred_path, cv2.IMREAD_GRAYSCALE)
        mask_img = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)

        pred_binary = (pred_img/255. >= th).astype(np.uint8)
        pred_binary = opening(pred_binary, 30)
        pred_binary = closing(pred_binary, 65)
        mask_flatten_list.extend((mask_img.flatten()/255.).astype(int))
        pred_flatten_list.extend(pred_binary.flatten())
    dice = fbeta_numpy(np.array(mask_flatten_list), np.array(pred_flatten_list), beta=0.5)
    print(f"th={th:.2f}, dice={dice:.4f}")

th=0.55, dice=0.5087
th=0.60, dice=0.5149
th=0.65, dice=0.5012
th=0.70, dice=0.4707
th=0.75, dice=0.4489


In [8]:
for th_ in range(55, 80, 5):
    th = th_ / 100
    pred_flatten_list = []
    mask_flatten_list = []
    for fold in [0,1,2,3,4]:
        pred_path = os.path.join(OUTPUT_DIR, "imgs", f"fold{fold}_average_slice555_valid_pred_img.png")
        mask_path = os.path.join(OUTPUT_DIR, "imgs", f"fold{fold}_average_slice555_valid_targets_img.png")
        pred_img = cv2.imread(pred_path, cv2.IMREAD_GRAYSCALE)
        mask_img = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)

        pred_binary = (pred_img/255. >= th).astype(np.uint8)
        pred_binary = opening(pred_binary, 65)
        pred_binary = closing(pred_binary, 65)
        mask_flatten_list.extend((mask_img.flatten()/255.).astype(int))
        pred_flatten_list.extend(pred_binary.flatten())
    dice = fbeta_numpy(np.array(mask_flatten_list), np.array(pred_flatten_list), beta=0.5)
    print(f"th={th:.2f}, dice={dice:.4f}")

th=0.55, dice=0.2301
th=0.60, dice=0.1615
th=0.65, dice=0.0722
th=0.70, dice=0.0097
th=0.75, dice=0.0000


In [9]:
for th_ in range(55, 80, 5):
    th = th_ / 100
    pred_flatten_list = []
    mask_flatten_list = []
    for fold in [0,1,2,3,4]:
        pred_path = os.path.join(OUTPUT_DIR, "imgs", f"fold{fold}_average_slice555_valid_pred_img.png")
        mask_path = os.path.join(OUTPUT_DIR, "imgs", f"fold{fold}_average_slice555_valid_targets_img.png")
        pred_img = cv2.imread(pred_path, cv2.IMREAD_GRAYSCALE)
        mask_img = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)

        pred_binary = (pred_img/255. >= th).astype(np.uint8)
        pred_binary = opening(pred_binary, 30)
        pred_binary = closing(pred_binary, 30)
        mask_flatten_list.extend((mask_img.flatten()/255.).astype(int))
        pred_flatten_list.extend(pred_binary.flatten())
    dice = fbeta_numpy(np.array(mask_flatten_list), np.array(pred_flatten_list), beta=0.5)
    print(f"th={th:.2f}, dice={dice:.4f}")

th=0.55, dice=0.5224
th=0.60, dice=0.5225
th=0.65, dice=0.5048
th=0.70, dice=0.4715
th=0.75, dice=0.4393


In [14]:
for th_ in range(30, 60, 5):
    th = th_ / 100
    pred_flatten_list = []
    mask_flatten_list = []
    for fold in [0,1,2,3,4]:
        pred_path = os.path.join(OUTPUT_DIR, "imgs", f"fold{fold}_average_slice555_valid_pred_img.png")
        mask_path = os.path.join(OUTPUT_DIR, "imgs", f"fold{fold}_average_slice555_valid_targets_img.png")
        pred_img = cv2.imread(pred_path, cv2.IMREAD_GRAYSCALE)
        mask_img = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)

        pred_binary = (pred_img/255. >= th).astype(np.uint8)
        pred_binary = opening(pred_binary, 30, 5)
        pred_binary = closing(pred_binary, 30, 5)
        mask_flatten_list.extend((mask_img.flatten()/255.).astype(int))
        pred_flatten_list.extend(pred_binary.flatten())
    dice = fbeta_numpy(np.array(mask_flatten_list), np.array(pred_flatten_list), beta=0.5)
    print(f"th={th:.2f}, dice={dice:.4f}")

th=0.30, dice=0.3985
th=0.35, dice=0.4400
th=0.40, dice=0.4616
th=0.45, dice=0.4695
th=0.50, dice=0.4752
th=0.55, dice=0.4406


In [15]:
for th_ in range(55, 80, 5):
    th = th_ / 100
    pred_flatten_list = []
    mask_flatten_list = []
    for fold in [0,1,2,3,4]:
        pred_path = os.path.join(OUTPUT_DIR, "imgs", f"fold{fold}_average_slice555_valid_pred_img.png")
        mask_path = os.path.join(OUTPUT_DIR, "imgs", f"fold{fold}_average_slice555_valid_targets_img.png")
        pred_img = cv2.imread(pred_path, cv2.IMREAD_GRAYSCALE)
        mask_img = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)

        pred_binary = (pred_img/255. >= th).astype(np.uint8)
        pred_binary = closing(pred_binary, 30, 2)
        pred_binary = opening(pred_binary, 30, 3)
        mask_flatten_list.extend((mask_img.flatten()/255.).astype(int))
        pred_flatten_list.extend(pred_binary.flatten())
    dice = fbeta_numpy(np.array(mask_flatten_list), np.array(pred_flatten_list), beta=0.5)
    print(f"th={th:.2f}, dice={dice:.4f}")

th=0.55, dice=0.5213
th=0.60, dice=0.5328
th=0.65, dice=0.5273
th=0.70, dice=0.5123
th=0.75, dice=0.4866
